# Custom Script with Pytorch

In [8]:
!yes | pip uninstall torchvision
!pip install -qU torchvision

yes: standard output: Broken pipe


In [2]:
!conda install torchvision -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::bokeh==1.4.0=py37_0
  - defaults/noarch::dask==2.11.0=py_0
  - defaults/linux-64::distributed==2.11.0=py37_0
  - defaults/linux-64::spyder==4.0.1=py37_0
  - defaults/linux-64::watchdog==0.10.2=py37_0
done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _pytorch_select-0.2        |            gpu_0           2 KB
    ca-certif

In [4]:
!conda install -c pytorch torchvision

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [9]:
!conda install pytorch torchvision torchaudio cpuonly -c pytorch -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cpuonly
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cpuonly-2.0                |                0           2 KB  pytorch
    ffmpeg-4.2.2               |       h20bf706_0        59.6 MB
    flit-core-3.6.0            |     pyhd3eb1b0_0          42 KB
    gnutls-3.6.15              |       he1e5248_0         1.0 MB
    lame-3.100                 |      

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/pytorch-mnist"

role = sagemaker.get_execution_role()

## Data
### Getting the data



In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms
import PIL

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    "pytorch-data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)


Extracting pytorch-data/MNIST/raw/train-images-idx3-ubyte.gz to pytorch-data/MNIST/raw




Extracting pytorch-data/MNIST/raw/train-labels-idx1-ubyte.gz to pytorch-data/MNIST/raw




Extracting pytorch-data/MNIST/raw/t10k-images-idx3-ubyte.gz to pytorch-data/MNIST/raw




Extracting pytorch-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to pytorch-data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: pytorch-data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [6]:

inputs = sagemaker_session.upload_data(path="pytorch-data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-866824485776/sagemaker/pytorch-mnist


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [7]:
!wget https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/main/Section-15-Custom-models/mnist-pytorch.py

--2023-07-05 15:52:15--  https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/main/Section-15-Custom-models/mnist-pytorch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8836 (8.6K) [text/plain]
Saving to: ‘mnist-pytorch.py’

mnist-pytorch.py    100%[===================>]   8.63K  --.-KB/s    in 0s      

2023-07-05 15:52:15 (69.4 MB/s) - ‘mnist-pytorch.py’ saved [8836/8836]



In [8]:
!pygmentize mnist-pytorch.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=sel

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c5.2xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist-pytorch.py` script above.


In [9]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist-pytorch.py",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
)

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [10]:
estimator.fit({"training": inputs})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-07-05-15-55-46-116


Using provided s3_resource
2023-07-05 15:55:46 Starting - Starting the training job...
2023-07-05 15:56:03 Starting - Preparing the instances for training......
2023-07-05 15:56:59 Downloading - Downloading input data...
2023-07-05 15:57:20 Training - Downloading the training image...
2023-07-05 15:58:10 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-05 11:58:24,092 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-05 11:58:24,094 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-05 11:58:24,096 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-05 11:58:24,106 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-05 11:58:24,108 sagemaker_pytorch_container.training INFO

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist-pytorch.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist-pytorch.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [11]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-866824485776/pytorch-training-2023-07-05-15-55-46-116/output/model.tar.gz), script artifact (s3://sagemaker-us-east-1-866824485776/pytorch-training-2023-07-05-15-55-46-116/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-866824485776/pytorch-training-2023-07-05-15-59-32-535/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-07-05-15-59-32-535
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-07-05-15-59-32-535
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-07-05-15-59-32-535


---------!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

In [12]:
!ls pytorch-data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [13]:
import gzip
import numpy as np
import random
import os

data_dir = "pytorch-data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=int)
data = images[mask]

In [14]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

Raw prediction result:
[[ -369.26565552 -1037.00732422 -1054.60131836  -644.28997803
   -602.12908936     0.          -465.54788208  -883.12597656
   -564.34851074  -543.80932617]
 [    0.          -724.17419434  -368.85925293  -232.40248108
   -469.56158447   -73.90075684  -166.74125671  -705.90576172
    -18.40313721  -494.55441284]
 [    0.         -1517.78991699  -951.49304199  -891.28240967
  -1221.64343262  -669.10986328  -825.09979248 -1036.35913086
   -859.21936035 -1080.99951172]
 [ -275.57363892  -438.57192993  -327.54400635  -167.3175354
   -410.69934082  -127.24650574  -287.60412598  -492.66424561
      0.          -371.19802856]
 [ -550.46472168     0.          -190.74188232  -152.35665894
   -263.68179321  -123.86851501  -211.76834106  -333.87979126
    -35.5320282   -214.67243958]
 [ -188.25378418  -899.94006348  -415.63546753  -756.39440918
   -618.15002441  -239.30383301     0.         -1087.40332031
   -523.06219482  -871.79302979]
 [ -751.56408691  -711.66082764  -72

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [15]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)

INFO:sagemaker:Deleting endpoint with name: pytorch-training-2023-07-05-15-59-32-535
